In [ ]:
import chess
import chess.engine
import random
import numpy 

starting_board = chess.Board()
starting_moves = ("h4 a5 Rh3 Ra6 Rd3 Rd6 Rd6 Nf6 h5 Nh5 Nc3 a4 Na4 Nc6 Rc6 Nf4 Ra6 Nh5 Ra8 Nf4 Rc8 Nh5 Ra8 Nf4 Ra5 Nh5 Nf3 Qa8 Nh4 Qa6 Nf3 Qh6 Ng5 Kd8 Nf3 Kc8 Ng5 Kb8 Nf3 Qg5 Rf5 Ka7 Nxg5 Ka6 Nf3 Nf6 Rf4 Ka5 Rb4 Ng4 Rb6 Nf6 Nc3 Nd5 Ne4 Ka4 Ra6+ Kb4 Ra8 Kc4 Rxf8 Rxf8 Ng3 Ra8 Ne5+ Kc5 Ng6 Ne3 Nf4 Nxf1 Ne6+ Kb6 Nf5 Ra4 Nd6 Ne3 Nf5 Nxf5 Kf1 Ne3+ Kg1 Nf5 Kh2 Ne3 Kh3 Nf5 Nd4 Nd6 Kg4 Ra3 Kg5 Rh3 Kf4 Rh1 Kg5 Rxd1 Kf4 Rxc1 Ke5 Rxa1 Nf5 Rh1 Nxd6 Kc6 Kf4 Kxd6 Kg3 Rh2 Kxh2 Kc6 Kh1 Kb6 Kg1 Ka7 Kf1 Ka8 Ke1 Kb8 Kd1 Kc8 Kc1 Kd8 Kb1 Ke8 Ka1 Kf8 Kb1 Kg8 Ka1 Kh8").split(" ")
for move in starting_moves:
    starting_board.push_san(move)

def new_board():
    return starting_board 
    

In [ ]:
board = new_board()
board

In [ ]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}

def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [ ]:
split_dims(board)

In [ ]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

def build_model_residual(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(board3d)
  for _ in range(conv_depth):
    previous = x
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, previous])
    x = layers.Activation('relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

model = build_model(32, 4)

In [ ]:
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	container = numpy.load('dataset.npz')
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
	return b, v


x_train, y_train = get_dataset()
x_train.transpose()
print(x_train.shape)
print(y_train.shape)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()
checkpoint_filepath = '/tmp/checkpoint/'
model_checkpointing_callback = ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_best_only= True,
)
model.fit(x_train, y_train,
          batch_size=2048,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4),model_checkpointing_callback])

model.save('model.h5')